In [1]:
## IMPORTS
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse

import time

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn import metrics
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.svm import SVC
import sklearn.feature_extraction
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import joblib

# Importing Shap for shapley values
import shap

from ordered_set import OrderedSet
from scipy.sparse import lil_matrix
from itertools import compress

c:\Users\tharu\anaconda3\envs\xai\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\tharu\anaconda3\envs\xai\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(all_masks,

In [2]:
# Import DataSet ds and Models
from src.datasets import IMDBDataset

ds = IMDBDataset(
    config_path="./configs/datasets/imdb.yaml", root="datasets/imdb", download=True
)
print(
    ds.x_test.shape,
    ds.x_train.shape,
    ds.x_val.shape,
    ds.y_test.shape,
    ds.y_train.shape,
    ds.y_val.shape,
)
print(
    type(ds.x_test),
    type(ds.x_train),
    type(ds.x_val),
    type(ds.y_test),
    type(ds.y_train),
    type(ds.y_val),
)

from src.models import AnalysisModels as Models

models = Models(
    config_path="./configs/models/analysis-models.yaml",
    root="./models/analysis-models",
    download=True,
)
print(models)

loaded_plain_model_rf = models.rf.model
loaded_plain_model_svc = models.svm.model
loaded_plain_model_lr = models.lr.model
loaded_plain_model_knn = models.knn.model
feature_names = ds.feature_names

## Preprocess text

x_train_imdb = ds.x_train
x_test_imdb = ds.x_test
x_val_imdb = ds.x_val

# Binarize y - Positive is 1
y_train_imdb = ds.y_train
y_test_imdb = ds.y_test
y_val_imdb = ds.y_val

Creating dataset
Initializing objects
Encoding
Dataset created
(4999, 11612) (40000, 11612) (5000, 11612) (4999,) (40000,) (5000,)
<class 'scipy.sparse._csr.csr_matrix'> <class 'scipy.sparse._csr.csr_matrix'> <class 'scipy.sparse._csr.csr_matrix'> <class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'numpy.ndarray'>
A collection of pretrained sklearn models.
Contains the models ['knn', 'lr', 'rf', 'svm']


In [ ]:
input_encoder = joblib.load("datasets/imdb/tfidf.pkl")
loaded_vocab = input_encoder.vocabulary_


## SEDC Function


In [4]:
def classifier_fn_lr(x, negative_to_positive=0):
    """Returns the prediction probability of class 1 -> Not class 0"""
    # print('loaded_plain_model_svc.decision_function(x) - ', loaded_plain_model_svc.decision_function(x))
    prediction = loaded_plain_model_lr.predict_proba(x)
    # If prediction is [1] retrurn the probability of class 1 else return probability of class 0
    if negative_to_positive == 1:
        return prediction[:, 0]
    return prediction[:, 1]

In [ ]:
# Do not need
# get the accuracy score of the model loaded_plain_model_lr
from sklearn.metrics import confusion_matrix

y_pred = loaded_plain_model_lr.predict(x_test_imdb)
accuracy = accuracy_score(y_test_imdb, y_pred)
print(f"Accuracy: {accuracy:.2f}")

cm = confusion_matrix(y_test_imdb, y_pred)

# Create a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

In [ ]:
coefficients = loaded_plain_model_lr.coef_
coefficients = coefficients.reshape(-1)
coefficients[0]


In [ ]:
def get_antonyms(word, model):
    """ " Get antonyms of a word and their indices in the feature vector
    Args:
        word: word to get antonyms for
        model: trained model with feature_importances_

    Returns:
        tuple of antonyms and their indices in the feature vector
    """
    antonyms = []
    antonyms_indices = []
    feature_importance = []
    temp_dict = {}
    for syn in wordnet.synsets(word):
        for i in syn.lemmas():
            if i.antonyms():
                antonyms.append(i.antonyms()[0].name())
    # Remove duplicates in antonyms
    antonyms = list(set(antonyms))

    for word in antonyms:
        if word in loaded_vocab:
            # antonyms_indices.append(ds.feature_names.tolist().index(word))
            # feature_importance.append(
            #     abs(coefficients[loaded_vocab[word]]))
            temp_dict[word] = abs(coefficients[loaded_vocab[word]])
    # Sort the antonyms and their indices based on feature importance
    # antonyms_indices = [x for _, x in sorted(
    #     zip(feature_importance, antonyms_indices), reverse=True)]
    # antonyms = [x for _, x in sorted(
    #     zip(feature_importance, antonyms), reverse=True)]
    # print(temp_dict)

    # return the key with the highest value
    if len(temp_dict) > 0:
        max_importance_idx = max(temp_dict, key=temp_dict.get)
        return [loaded_vocab[max_importance_idx]]
    else:
        return []

    # print(antonyms)
    # print(feature_importance)
    # if len(feature_importance) > 0:
    #     max_importance_idx = np.argmax(feature_importance)
    #     return [antonyms_indices[max_importance_idx]]
    # else:
    #     return []

    # if len(antonyms_indices) > 0:
    #     return [antonyms_indices[0]]
    # else:
    #     return []

In [ ]:
def perturb_fn(x, inst, print_flag=0):
    """Function to perturb instance x -> Deform the array -> assign 0 to the x-th column"""
    """
    Returns perturbed instance inst
    """
    inst[:, x] = 0
    return inst


def replace_fn(x, y, inst, print_flag=0):
    """Function to perturb instance x -> Deform the array -> assign 0 to the x-th column"""
    """
    Returns perturbed instance inst
    """
    new_inst = inst.copy()
    try:
        temp_x = inst[:, x]
        temp_y = inst[:, y]
        new_inst[:, x] = temp_y
        new_inst[:, y] = temp_x
    except:
        new_inst[:, x] = 0
    return new_inst


def conditional_replace_fn(x, y, inst, print_flag=0):
    for i in range(len(x)):
        if isinstance(y[i], str):
            inst[:, x[i]] = 0
        else:
            temp_x = inst[:, x[i]]
            temp_y = inst[:, y[i]]
            inst[:, x[i]] = temp_y
            inst[:, y[i]] = temp_x
    return inst


def print_instance(pert_inst, ref_inst, feature_names):
    """Function to print the perturbed instance"""
    """
    Returns perturbed instance inst
    """
    indices_active_elements_ref = np.nonzero(ref_inst)[1]
    indices_active_elements_pert = np.nonzero(pert_inst)[1]
    ref_set = set(indices_active_elements_ref)
    pert_set = set(indices_active_elements_pert)
    # elements in ref_set but not in pert_set
    removed_word_indices = ref_set - pert_set
    # elements in pert_set but not in ref_set
    added_word_indices = pert_set - ref_set
    printable_array = []
    for item in indices_active_elements_ref:
        printable_array.append(".." + feature_names[item] + "..")
    # Change formatting of removed words
    for item in removed_word_indices:
        printable_array[printable_array.index(".." + feature_names[item] + "..")] = (
            "--" + feature_names[item] + "--"
        )
    # change formatting of added words
    for item in added_word_indices:
        printable_array.append("++" + feature_names[item] + "++")
    printable_array.append(classifier_fn_lr(pert_inst))
    print(printable_array)
    return printable_array


def print_ref_instance(ref_inst, feaaure_names):
    printable_array = []
    indices_active_elements = np.nonzero(ref_inst)[1]
    for item in indices_active_elements:
        printable_array.append(".." + feature_names[item] + "..")
    print(printable_array)


"""
Input:
    - comb: "best-first" (combination of) feature(s) that is expanded
    (e.g., comb_to_expand)
    - expanded_combis: list of combinations of features that are already 
    expanded as "best-first"
    - feature_set: indices of the active features of the instance 
    - candidates_to_expand: combinations of features that are candidates to be 
    expanded in next iterations or candidates for "best-first"
    - explanations_sets: counterfactual explanations already found
    - scores_candidates_to_expand: scores after perturbation for the candidate
    combinations of features to be expanded
    - instance: instance to be explained
    - cf: classifier prediction probability function
    or decision function. For ScikitClassifiers, this is classifier.predict_proba 
    or classifier.decision_function or classifier.predict_log_proba.
    Make sure the function only returns one (float) value. For instance, if you
    use a ScikitClassifier, transform the classifier.predict_proba as follows:

        def classifier_fn(X):
            c=classification_model.predict_proba(X)
            y_predicted_proba=c[:,1]
            return y_predicted_proba

Returns:
    - explanation_candidates: combinations of features that are explanation
    candidates to be checked in the next iteration
    - candidates_to_expand: combinations of features that are candidates to be 
    expanded in next iterations or candidates for "best-first"
    - expanded_combis: [list] list of combinations of features that are already 
    expanded as "best-first" 
    - scores_candidates_to_expand: scores after perturbation for the candidate
    combinations of features to be expanded
    - scores_explanation_candidates: scores after perturbation of explanation candidates
"""


def expand_and_prune(
    comb,
    replacement_comb_to_expand,
    expanded_combis,
    feature_set,
    candidates_to_expand,
    candidates_to_expand_replacements,
    explanations_sets,
    explanation_replacement_sets,
    scores_candidates_to_expand,
    instance,
    cf,
    revert=0,
    replacements=[],
):
    """Function to expand "best-first" feature combination and prune explanation_candidates and candidates_to_expand"""

    comb = OrderedSet(comb)
    replacement_comb_to_expand = OrderedSet(replacement_comb_to_expand)
    print("comb: ", comb)
    print("replacement_comb_to_expand: ", replacement_comb_to_expand)
    expanded_combis.append(comb)

    old_candidates_to_expand = [frozenset(x) for x in candidates_to_expand]
    old_candidates_to_expand = set(old_candidates_to_expand)
    print("feature_set: ", feature_set)
    feature_set_new = []
    feature_set_new_replacements = []
    ## If the feature is not in the current combination -> add it to a new list
    for feature in feature_set:
        list_feature = list(feature)
        if len(comb & feature) == 0:  # set operation: intersection
            replacement_feature = get_antonyms(
                feature_names[list_feature[0]], loaded_plain_model_rf
            )
            replacement_feature = frozenset(replacement_feature)
            if replacement_feature == frozenset():
                new_string = "0" * (len(comb) + 1)
                replacement_feature = frozenset([new_string])
            # print("replacement_feature: ", replacement_feature, "feature: ", feature)
            feature_set_new.append(
                feature
            )  # If the feature is not in the current combination to remove from the instance
            feature_set_new_replacements.append(replacement_feature)

    print("feature_set_new: ", feature_set_new)
    print("feature_set_new_replacements: ", feature_set_new_replacements)
    # Add each element in the new set -> which were initially not present -> to the accepted combination -> create new combinations -> (EXPANSION)
    new_explanation_candidates = []
    new_explanation_candidates_replacements = []
    # for element in feature_set_new:
    #     union = (comb|element) #set operation: union
    #     union_replacements = (replacement_comb_to_expand|feature_set_new_replacements[i])
    #     new_explanation_candidates.append(union) # Create new combinations to remove from the instance
    #     new_explanation_candidates_replacements.append(union_replacements)

    for i in range(len(feature_set_new)):
        union = comb | feature_set_new[i]
        union_replacements = (
            replacement_comb_to_expand | feature_set_new_replacements[i]
        )
        new_explanation_candidates.append(
            union
        )  # Create new combinations to remove from the instance
        new_explanation_candidates_replacements.append(union_replacements)

    print("new_explanation_candidates: ", new_explanation_candidates)
    print(
        "new_explanation_candidates_replacements: ",
        new_explanation_candidates_replacements,
    )

    # Add new explanation candidates to the list of candidates to expand
    candidates_to_expand_notpruned = candidates_to_expand.copy()
    candidates_to_expand_replacements_notpruned = (
        candidates_to_expand_replacements.copy()
    )
    # for new_candidate in new_explanation_candidates:
    #     candidates_to_expand_notpruned.append(new_candidate)
    for i in range(len(new_explanation_candidates_replacements)):
        candidates_to_expand_notpruned.append(new_explanation_candidates[i])
        candidates_to_expand_replacements_notpruned.append(
            new_explanation_candidates_replacements[i]
        )

    print("candidates_to_expand_notpruned: ", candidates_to_expand_notpruned)
    print(
        "candidates_to_expand_replacements_notpruned: ",
        candidates_to_expand_replacements_notpruned,
    )

    # Calculate scores of new combinations and add to scores_candidates_to_expand
    # perturb each new candidate and get the score for each.
    # perturbed_instances = [perturb_fn(x, inst=instance.copy(), print_flag=1) for x in new_explanation_candidates]
    replaced_instances = []
    for i in range(len(new_explanation_candidates)):
        #     #print("i: ", i, "new_explanation_candidates[i]: ", new_explanation_candidates[i], "new_explanation_candidates_replacements[i]: ", new_explanation_candidates_replacements[i])
        #     if isinstance(new_explanation_candidates_replacements[i][0], int):
        #         replaced_instances.append(perturb_fn(x=new_explanation_candidates[i], inst=instance.copy()))
        #     else:
        replaced_instances.append(
            conditional_replace_fn(
                x=new_explanation_candidates[i],
                y=new_explanation_candidates_replacements[i],
                inst=instance.copy(),
                print_flag=1,
            )
        )
    # -------------------------------------------
    print("len(perturbed_instances): ", len(replaced_instances))
    perturbed_instances = replaced_instances

    # word_sets = []
    # replacement_word_sets = []
    # for item in new_explanation_candidates:
    #     item_word = []
    #     word_replacement = []
    #     for index in item:
    #         item_word.append(feature_names[index])
    #         word_replacement.append(get_antonyms(feature_names[index], loaded_plain_model_rf))
    #     word_sets.append(item_word)
    #     replacement_word_sets.append(word_replacement)
    # print(word_sets)
    # print(replacement_word_sets)
    # #replacements = np.array(replacement_word_sets).reshape(len(replacement_word_sets), 1)
    # replacements = []
    # # for features in replacement_word_sets:
    # #     replacements.append(OrderedSet(features))
    # # replacements = [frozenset(x) for x in replacements]

    # replaced_instances = []
    # perturbed_instances = [perturb_fn(x, inst=instance.copy()) for x in new_explanation_candidates]
    print("Expanded sentences from the above chosen combination")
    for item in perturbed_instances:
        print_instance(item, instance.copy(), feature_names)
    scores_perturbed_new = [cf(x, revert) for x in perturbed_instances]
    ## Append the newly created score array to the passes existing array
    scores_candidates_to_expand_notpruned = (
        scores_candidates_to_expand + scores_perturbed_new
    )
    # create a dictionary of scores dictionary where the
    # keys are string representations of the candidates from candidates_to_expand_notpruned, and the
    # values are the corresponding scores from scores_candidates_to_expand_notpruned
    dictionary_scores = dict(
        zip(
            [str(x) for x in candidates_to_expand_notpruned],
            scores_candidates_to_expand_notpruned,
        )
    )

    # *** Pruning step: remove all candidates to expand that have an explanation as subset ***
    candidates_to_expand_pruned_explanations = []
    candidates_to_expand_pruned_replacements_explanations = []
    # take one combination from candidates
    # Rewritten using list indices
    # for combi in candidates_to_expand_notpruned:
    #     pruning=0
    #     for explanation in explanations_sets: # if an explanation is present as a subser in combi, does not add it to the to be expanded list -> because solution with a smaller size exists
    #         if ((explanation.issubset(combi)) or (explanation==combi)):
    #             pruning = pruning + 1
    #     if (pruning == 0): # If it is not a superset of a present explanation -> add it to the list
    #         candidates_to_expand_pruned_explanations.append(combi)
    # Write the above function using list indices
    for i in range(len(candidates_to_expand_notpruned)):
        pruning = 0
        for explanation in explanations_sets:
            if (explanation.issubset(candidates_to_expand_notpruned[i])) or (
                explanation == candidates_to_expand_notpruned[i]
            ):
                pruning = pruning + 1
        if pruning == 0:
            candidates_to_expand_pruned_explanations.append(
                candidates_to_expand_notpruned[i]
            )
            candidates_to_expand_pruned_replacements_explanations.append(
                candidates_to_expand_replacements_notpruned[i]
            )

    # Each element is frozen as a set
    candidates_to_expand_pruned_explanations_frozen = [
        frozenset(x) for x in candidates_to_expand_pruned_explanations
    ]
    candidates_to_expand_pruned_replacements_explanations_frozen = [
        frozenset(x) for x in candidates_to_expand_pruned_replacements_explanations
    ]

    # But the total set f frozen sets are not frozen
    candidates_to_expand_pruned_explanations_ = set(
        candidates_to_expand_pruned_explanations_frozen
    )
    candidates_to_expand_pruned_replacements_explanations_ = set(
        candidates_to_expand_pruned_replacements_explanations_frozen
    )

    expanded_combis_frozen = [frozenset(x) for x in expanded_combis]
    expanded_combis_ = set(expanded_combis_frozen)

    # *** Pruning step: remove all candidates to expand that are in expanded_combis *** -> Same as above
    candidates_to_expand_pruned = (
        candidates_to_expand_pruned_explanations_ - expanded_combis_
    )
    candidates_to_expand_pruned_replacements = (
        candidates_to_expand_pruned_replacements_explanations_ - expanded_combis_
    )
    ind_dict = dict(
        (k, i) for i, k in enumerate(candidates_to_expand_pruned_explanations_frozen)
    )
    indices = [ind_dict[x] for x in candidates_to_expand_pruned]
    candidates_to_expand = [
        candidates_to_expand_pruned_explanations[i] for i in indices
    ]
    candidates_to_expand_replacements = [
        candidates_to_expand_pruned_replacements_explanations[i] for i in indices
    ]

    # The new explanation candidates are the ones that are NOT in the old list of candidates to expand
    new_explanation_candidates_pruned = (
        candidates_to_expand_pruned - old_candidates_to_expand
    )
    candidates_to_expand_frozen = [frozenset(x) for x in candidates_to_expand]
    candidates_to_expand_replacements_frozen = [
        frozenset(x) for x in candidates_to_expand_replacements
    ]

    ind_dict2 = dict((k, i) for i, k in enumerate(candidates_to_expand_frozen))
    indices2 = [ind_dict2[x] for x in new_explanation_candidates_pruned]
    explanation_candidates = [candidates_to_expand[i] for i in indices2]
    explanation_candidates_replacements = [
        candidates_to_expand_replacements[i] for i in indices2
    ]

    # Get scores of the new candidates and explanations.
    scores_candidates_to_expand = [
        dictionary_scores[x] for x in [str(c) for c in candidates_to_expand]
    ]
    scores_explanation_candidates = [
        dictionary_scores[x] for x in [str(c) for c in explanation_candidates]
    ]

    return (
        explanation_candidates,
        explanation_candidates_replacements,
        candidates_to_expand,
        candidates_to_expand_replacements,
        expanded_combis,
        scores_candidates_to_expand,
        scores_explanation_candidates,
    )

In [ ]:
class SEDC_Explainer(object):
    """Class for generating evidence counterfactuals for classifiers on behavioral/text data"""

    def __init__(
        self,
        feature_names,
        classifier_fn,
        threshold_classifier,
        max_iter=100,
        max_explained=1,
        BB=True,
        max_features=30,
        time_maximum=120,
        revert=0,
    ):
        """Init function

        Args:
            classifier_fn: [function] classifier prediction probability function
            or decision function. For ScikitClassifiers, this is classifier.predict_proba
            or classifier.decision_function or classifier.predict_log_proba.
            Make sure the function only returns one (float) value. For instance, if you
            use a ScikitClassifier, transform the classifier.predict_proba as follows:

                def classifier_fn(X):
                    c=classification_model.predict_proba(X)
                    y_predicted_proba=c[:,1]
                    return y_predicted_proba

            threshold_classifier: [float] the threshold that is used for classifying
            instances as positive or not. When score or probability exceeds the
            threshold value, then the instance is predicted as positive.
            We have no default value, because it is important the user decides
            a good value for the threshold.

            feature_names: [numpy.array] contains the interpretable feature names,
            such as the words themselves in case of document classification or the names
            of visited URLs.

            max_iter: [int] maximum number of iterations in the search procedure.
            Default is set to 50.

            max_explained: [int] maximum number of EDC explanations generated.
            Default is set to 1.

            BB: [“True” or “False”]  when the algorithm is augmented with
            branch-and-bound (BB=True), one is only interested in the (set of)
            shortest explanation(s). Default is "True".

            max_features: [int] maximum number of features allowed in the explanation(s).
            Default is set to 30.

            time_maximum: [int] maximum time allowed to generate explanations,
            expressed in minutes. Default is set to 2 minutes (120 seconds).
        """

        self.feature_names = feature_names
        self.classifier_fn = classifier_fn
        self.threshold_classifier = threshold_classifier
        self.max_iter = max_iter
        self.max_explained = max_explained
        self.BB = BB
        self.max_features = max_features
        self.time_maximum = time_maximum
        self.revert = None
        self.initial_class = None
        self._report_data = {}

    def explanation(self, instance):
        """Generates evidence counterfactual explanation for the instance.
        ONLY IF THE CURRENT INSTANCE IS POSITIVE -> Limitation

        Args:
            instance: [numpy.array or sparse matrix] instance to explain

        Returns:
            A dictionary where:

                explanation_set: explanation(s) ranked from high to low change
                in predicted score or probability.
                The number of explanations shown depends on the argument max_explained.

                number_active_elements: number of active elements of
                the instance of interest.

                number_explanations: number of explanations found by algorithm.

                minimum_size_explanation: number of features in the smallest explanation.

                time_elapsed: number of seconds passed to generate explanation(s).

                explanations_score_change: change in predicted score/probability
                when removing the features in the explanation, ranked from
                high to low change.
        """

        # *** INITIALIZATION ***
        print("Start initialization...")
        tic = time.time()
        instance = lil_matrix(instance)
        print("initial sentence is ... ")
        print(instance.get_shape())
        print_ref_instance(instance, self.feature_names)
        iteration = 0
        nb_explanations = 0
        minimum_size_explanation = np.nan
        explanations = []
        explanations_replacements = []
        explanations_sets = []
        explanation_replacement_sets = []
        explanations_score_change = []
        expanded_combis = []
        score_predicted = self.classifier_fn(instance)  ## Returns Prediction Prob
        # Intial class is 1 is score is greater than threshold
        if score_predicted > self.threshold_classifier:
            self.initial_class = [1]
        else:
            self.initial_class = [0]
            self.revert = 1
        print(
            "score_predicted  ",
            score_predicted,
            "  initial_class  ",
            self.initial_class,
        )

        reference = np.reshape(
            np.zeros(np.shape(instance)[1]), (1, len(np.zeros(np.shape(instance)[1])))
        )
        reference = sparse.csr_matrix(reference)

        # explainer = shap.KernelExplainer(self.classifier_fn, reference, link="identity")
        # shapVals = explainer.shap_values(instance, nsamples=5000, l1_reg="aic")

        # features = []
        # for ind in range(len(shapVals[0])):
        #     if shapVals[0, ind] != 0:
        #         features.append({"feature": ind, "shapValue": shapVals[0, ind]})
        # sorted_data_in = sorted(features, key=lambda x: x["shapValue"], reverse=True)
        # inverse_sorted_data_in = sorted(features, key=lambda x: x["shapValue"])

        # if self.revert == 1:
        #     sorted_data_in = inverse_sorted_data_in

        indices_active_elements = np.nonzero(instance)[
            1
        ]  ## -> Gets non zero elements in the instance as an array [x, y, z]
        # sorted_indices = sorted(
        #     indices_active_elements, key=lambda x: shapVals[0, x], reverse=True
        # )
        # indices_active_elements = np.array(sorted_indices)
        number_active_elements = len(indices_active_elements)
        indices_active_elements = indices_active_elements.reshape(
            (number_active_elements, 1)
        )  ## -> Reshape to get a predictable

        candidates_to_expand = (
            []
        )  # -> These combinations are further expanded -> These are the elements to be removed from the sentence
        for features in indices_active_elements:
            candidates_to_expand.append(OrderedSet(features))
        ## > Gets an array with each element in reshaped incides as an ordered set -> [OrderedSet([430]), OrderedSet([588]), OrderedSet([595])]
        candidates_to_expand_replacements = []
        for features in indices_active_elements:
            candidates_to_expand_replacements.append(
                OrderedSet(
                    get_antonyms(self.feature_names[features[0]], loaded_plain_model_rf)
                )
            )
        for i in range(len(candidates_to_expand_replacements)):
            if candidates_to_expand_replacements[i] == OrderedSet():
                candidates_to_expand_replacements[i] = OrderedSet(["0"])
        explanation_candidates = candidates_to_expand.copy()
        explanation_candidates_replacements = candidates_to_expand_replacements.copy()
        print("explanation_candidates /n", explanation_candidates)
        print(
            "explanation_candidates_replacements /n",
            explanation_candidates_replacements,
        )
        ## Gets a copy of the above array -> Initially

        feature_set = [
            frozenset(x) for x in indices_active_elements
        ]  ## Immutable -> can be used as keys in dictionary
        ## Used features in the current x-reference -> incides of the words in the review.

        print("Initialization is complete.")
        print("\n Elapsed time %d \n" % (time.time() - tic))

        # *** WHILE LOOP ***
        while (
            (iteration < self.max_iter)
            and (nb_explanations < self.max_explained)
            and (len(candidates_to_expand) != 0)
            and (len(explanation_candidates) != 0)
            and ((time.time() - tic) < self.time_maximum)
        ):
            ## Stop if maximum iterations exceeded
            #  number of explanations generated is greater than the maximum explanations
            #  There are no candidates to expand
            #  There are no explanation candidates -> Used to force stop while loop below
            #  Or maximum allowed time exceeded
            iteration += 1
            print("\n Iteration %d \n" % iteration)

            if iteration == 1:
                print("Run in first iteration -> perturbation done \n")
                # Print the word in each index in the explanation candidates
                # for item in explanation_candidates:
                #     print([self.feature_names[x] for x in item])
                replacements = [
                    get_antonyms(self.feature_names[x[0]], loaded_plain_model_rf)
                    for x in explanation_candidates
                ]
                # convert each element in replacement to a OrderedSet
                replacements = explanation_candidates_replacements
                print("replacements \n", replacements, "\n")
                print("explanation_candidates \n", explanation_candidates, "\n")
                perturbed_instances = []
                print("After changing or removing words,  ")
                replaced_instances = []
                for i in range(len(explanation_candidates)):
                    if replacements[i] == OrderedSet(["0"]):
                        replaced_instances.append(
                            perturb_fn(
                                x=explanation_candidates[i], inst=instance.copy()
                            )
                        )
                    else:
                        replaced_instances.append(
                            replace_fn(
                                x=explanation_candidates[i],
                                y=replacements[i],
                                inst=instance.copy(),
                            )
                        )
                # Remove the elements in the indices given by the ordered set x and return an array fo such elements
                # Removes only one element in the first run -> Contains sentences with one word removed
                perturbed_instances = replaced_instances
                for instance_p in perturbed_instances:
                    print_instance(instance_p, instance, self.feature_names)
                scores_explanation_candidates = [
                    self.classifier_fn(x, self.revert) for x in perturbed_instances
                ]
                # Get predictions for each perturbed instance where one or more elements are removed from the initial instance
                # It is in form of [[x], [y], [z]]
                print(
                    "scores_explanation_candidates \n",
                    scores_explanation_candidates,
                    "\n",
                )
                scores_candidates_to_expand = scores_explanation_candidates.copy()

            scores_perturbed_new_combinations = [
                x[0] for x in scores_explanation_candidates
            ]
            # Therefore get it to the shape [x, y, z] by getting the [0] th element of each element array
            print(
                "scores_perturbed_new_combinations ", scores_perturbed_new_combinations
            )

            # ***CHECK IF THERE ARE EXPLANATIONS***
            new_explanations = list(
                compress(
                    explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            new_explanation_replacements = list(
                compress(
                    explanation_candidates_replacements,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            # Get explanation candidates where their probability is less than the threshold classifier -> Positive becomes negative
            # print("New Explanations \n", new_explanations)
            explanations += list(
                compress(
                    explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            explanations_replacements += list(
                compress(
                    explanation_candidates_replacements,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            # print("\n explanations, explanations_score_change", explanations)
            nb_explanations += len(
                list(
                    compress(
                        explanation_candidates,
                        scores_perturbed_new_combinations < self.threshold_classifier,
                    )
                )
            )  # Update number of explanations which pass the required threshold
            explanations_sets += list(
                compress(
                    explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            explanation_replacement_sets += list(
                compress(
                    explanation_candidates_replacements,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            explanations_sets = [
                set(x) for x in explanations_sets
            ]  # Convert each array to a set -> to get the words
            explanation_replacement_sets = [
                set(x) for x in explanation_replacement_sets
            ]
            explanations_score_change += list(
                compress(
                    scores_explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            print("explanations_score_change", explanations_score_change)

            # Adjust max_length
            if self.BB == True:
                if len(explanations) != 0:
                    lengths = []  # Record length of each explanation found
                    for explanation in explanations:
                        lengths.append(len(explanation))
                    lengths = np.array(lengths)
                    max_length = lengths.min()
                    # Get minimum length of the found explanations as max length -> Do not search for explanations with longer length
                else:
                    max_length = number_active_elements  # Else can find maximum length equal to number of words in instance
            else:
                max_length = number_active_elements
            print("\n-------------Max length updated to - ", max_length)

            # Eliminate combinations from candidates_to_expand ("best-first" candidates) that can not be expanded
            # Pruning based on Branch & Bound=True, max. features allowed and number of active features
            candidates_to_expand_updated = []
            candidates_to_expand_updated_replacements = []
            scores_candidates_to_expand_updated = (
                []
            )  # enumerate -> Find count of || to list one after another
            for j, combination in enumerate(candidates_to_expand):
                if (
                    (len(combination) < number_active_elements)
                    and (len(combination) < max_length)
                    and (len(combination) < self.max_features)
                ):
                    # Combination length should be less than the words in the input and max length of the required explanation and required maximum features
                    candidates_to_expand_updated.append(
                        combination
                    )  # If the combination matches, it is further expanded
                    scores_candidates_to_expand_updated.append(
                        scores_candidates_to_expand[j]
                    )
                    # Add the prediction score to the new array
                    # get the score from the scores_candidates_to_expand using the current index
                    candidates_to_expand_updated_replacements.append(
                        candidates_to_expand_replacements[j]
                    )
                    # Add the replacement to the new array

            print(
                "\nlen(candidates_to_expand_updated)",
                len(candidates_to_expand_updated),
                " 0 ",
            )
            print(
                "\nnb_explanations",
                nb_explanations,
                " >= self.max_explained ",
                self.max_explained,
            )

            # *** IF LOOP ***
            # expanding the candidates to update will exceed the max length set in the earlier loop
            if (len(candidates_to_expand_updated) == 0) or (
                nb_explanations >= self.max_explained
            ):
                ## If the number of explanations exceeded the required number
                ## or no candidates
                ## no explanations present

                print("nb_explanations Stop iterations...")
                explanation_candidates = []  # stop algorithm
                ## Found all the candidates
                print(
                    "scores_candidates_to_expand_updated  ",
                    scores_candidates_to_expand_updated,
                )
                # print("candidates_to_expand_updated   ", candidates_to_expand_updated)

            elif len(candidates_to_expand_updated) != 0:
                ## If there are possible candidates

                explanation_candidates = []
                it = 0  # Iteration of the while loop
                indices = []

                scores_candidates_to_expand2 = []
                for score in scores_candidates_to_expand_updated:
                    if score[0] < self.threshold_classifier:
                        scores_candidates_to_expand2.append(2 * score_predicted)
                    else:
                        scores_candidates_to_expand2.append(score)
                # update candidate scores if they have score less than threshold -> To expand them further
                # shap_candidates_to_expand2 = []
                # for candidate in candidates_to_expand_updated:
                #     shapValues = 0
                #     for word in candidate:
                #         # find word in feature column in sorted_data
                #         for ind in range(len(sorted_data_in)):
                #             if sorted_data_in[ind]["feature"] == word:
                #                 shapValues += sorted_data_in[ind]["shapValue"]
                #                 break
                #     shap_candidates_to_expand2.append(shapValues)

                print(
                    "\n scores_candidates_to_expand2 before loop",
                    scores_candidates_to_expand2,
                )
                print(
                    len(explanation_candidates),
                    it,
                    "<",
                    len(scores_candidates_to_expand2),
                )

                # *** WHILE LOOP ***
                while (
                    (len(explanation_candidates) == 0)
                    and (it < len(scores_candidates_to_expand2))
                    and ((time.time() - tic) < self.time_maximum)
                ):
                    # Stop if candidates are found or looped through more than there are candidates or maximum time reached

                    print("While loop iteration %d" % it)

                    if it != 0:  # Because indices are not there in the first iteration
                        for index in indices:
                            scores_candidates_to_expand2[index] = 2 * score_predicted

                    # print(
                    #     "\n scores_candidates_to_expand2 after loop",
                    #     scores_candidates_to_expand2,
                    # )
                    # print("\n indices", indices)

                    # do elementwise subtraction between score_predicted and scores_candidates_to_expand2
                    subtractionList = []
                    for item in scores_candidates_to_expand2:
                        subtractionList.append(item - score_predicted)
                    # for x, y in zip(score_predicted, scores_candidates_to_expand2):
                    #     subtractionList.append(x - y)
                    # print("subtractionList", subtractionList)

                    # Do element wise subtraction between the prediction score of the x_ref and every element of the scores_candidates_to_expand2
                    index_combi_max = np.argmax(subtractionList)
                    # index_shap_max = np.argmax(shap_candidates_to_expand2)
                    # index_shap_min = np.argmin(shap_candidates_to_expand2)
                    if self.revert == 0:
                        index_combi_max = np.argmax(subtractionList)
                    else:
                        index_combi_max = np.argmin(subtractionList)
                    # if self.revert == 0:
                    #     index_combi_max = index_shap_max
                    # else:
                    #     index_combi_max = index_shap_min
                    # print(
                    #     "subtrac max ",
                    #     index_combi_max,
                    #     " index_shap_max ",
                    #     index_shap_max,
                    # )
                    # Get the index of the maximum value -> Expand it
                    print(
                        "\n index_combi_max",
                        candidates_to_expand_updated[np.argmax(subtractionList)],
                    )
                    indices.append(index_combi_max)
                    expanded_combis.append(
                        candidates_to_expand_updated[index_combi_max]
                    )
                    # Add this combination to already expanded combinations as it will be expanded next by expand and prune function

                    comb_to_expand = candidates_to_expand_updated[index_combi_max]
                    replacement_comb_to_expand = (
                        candidates_to_expand_updated_replacements[index_combi_max]
                    )
                    words_comb_selected = []
                    for item in candidates_to_expand_updated[index_combi_max]:
                        words_comb_selected.append(feature_names[item])
                    print("The chosen combination is ", words_comb_selected)
                    print_instance(
                        conditional_replace_fn(
                            candidates_to_expand_updated[index_combi_max],
                            candidates_to_expand_updated_replacements[index_combi_max],
                            instance.copy(),
                        ),
                        instance,
                        self.feature_names,
                    )
                    print(
                        "It has a score of ",
                        scores_candidates_to_expand_updated[index_combi_max],
                    )
                    # Expand the found combination with highest difference
                    print("comb_to_expand", comb_to_expand)
                    print("replacement_comb_to_expand", replacement_comb_to_expand)
                    func = expand_and_prune(
                        comb_to_expand,
                        replacement_comb_to_expand,
                        expanded_combis,
                        feature_set,
                        candidates_to_expand_updated,
                        candidates_to_expand_updated_replacements,
                        explanations_sets,
                        explanation_replacement_sets,
                        scores_candidates_to_expand_updated,
                        instance,
                        self.classifier_fn,
                        self.revert,
                        replacements,
                    )
                    """Returns:
                        - explanation_candidates: combinations of features that are explanation
                        candidates to be checked in the next iteration
                        - candidates_to_expand: combinations of features that are candidates to
                        expanded in next iterations or candidates for "best-first"
                        - expanded_combis: [list] list of combinations of features that are already
                        expanded as "best-first"
                        - scores_candidates_to_expand: scores after perturbation for the candidate
                        combinations of features to be expanded
                        - scores_explanation_candidates: scores after perturbation of explanation candidates"""
                    explanation_candidates = func[0]
                    explanation_candidates_replacements = func[1]
                    candidates_to_expand = func[2]
                    candidates_to_expand_replacements = func[3]
                    expanded_combis = func[4]
                    scores_candidates_to_expand = func[5]
                    scores_explanation_candidates = func[6]

                    it += 1

                print(
                    "\n\n\niteration - ", iteration, " self.max_iter - ", self.max_iter
                )
                print(
                    "\n\nlen(candidates_to_expand) - ",
                    len(candidates_to_expand),
                    " != 0 ",
                )
                print(
                    "\n\nlen(explanation_candidates) - ",
                    len(explanation_candidates),
                    " !=0 ",
                )
                print(
                    "\n\n(time.time() - tic) - ",
                    (time.time() - tic),
                    " self.time_maximum - ",
                    self.time_maximum,
                )
            print("\n Elapsed time %d \n" % (time.time() - tic))

        # *** FINAL PART OF ALGORITHM ***
        print("Iterations are done.")

        explanation_set = []
        explanation_feature_names = []
        index_of_min_length_explanation = -1
        for i in range(len(explanations)):
            explanation_feature_names = []
            for features in explanations[i]:
                explanation_feature_names.append(self.feature_names[features])
            explanation_set.append(explanation_feature_names)

        if len(explanations) != 0:
            lengths_explanation = []
            for explanation in explanations:
                l = len(explanation)
                lengths_explanation.append(l)
            minimum_size_explanation = np.min(lengths_explanation)
            index_of_min_length_explanation = np.argmin(lengths_explanation)
        try:
            print("argmin", explanations[index_of_min_length_explanation])
        except:
            pass

        print("Final sentence")
        final_sentence = conditional_replace_fn(
            explanations[index_of_min_length_explanation],
            explanations_replacements[index_of_min_length_explanation],
            instance.copy(),
        )
        final_sentence = print_instance(final_sentence, instance, self.feature_names)
        new_instance = instance.copy()
        new_replacements = []
        replacement_features = []
        for feature in explanations[index_of_min_length_explanation]:
            feature_replacement = get_antonyms(
                feature_names[feature], loaded_plain_model_rf
            )
            print("feature_replacement", feature_replacement)
            new_replacements.append(feature_replacement)
        print("new_replacements", new_replacements)
        print("replacementfeature", feature_names[new_replacements[0]])

        output_removed_words = []
        for item in explanations[index_of_min_length_explanation]:
            output_removed_words.append(feature_names[item])
        try:
            replacementWords = []
            for item_ind in range(len(new_replacements)):
                replacementWords.append(
                    {
                        "feature": feature_names[
                            explanations[index_of_min_length_explanation][item_ind]
                        ],
                        "replacement": feature_names[new_replacements[item_ind]][0],
                    }
                )
            print("replacementWords", replacementWords)
        except:
            pass

        new_insatnce = instance.copy()
        index_of_min_length_explanation = -1
        for relpacement_feature_index in range(len(new_replacements)):
            if new_replacements[relpacement_feature_index] != []:
                new_insatnce = replace_fn(
                    x=explanations[index_of_min_length_explanation][
                        relpacement_feature_index
                    ],
                    y=new_replacements[relpacement_feature_index],
                    inst=new_insatnce,
                )
                replacement_features.append(
                    feature_names[new_replacements[relpacement_feature_index]]
                )
            else:
                new_insatnce = perturb_fn(
                    explanations[index_of_min_length_explanation][
                        relpacement_feature_index
                    ],
                    new_insatnce,
                )
                replacement_features.append(feature_names[relpacement_feature_index])
        final_prob = self.classifier_fn(new_insatnce)
        print("final_prob", final_prob)

        final_exp = []
        for i in range(len(explanations[index_of_min_length_explanation])):
            if new_replacements[i] != []:
                final_exp.append(
                    [output_removed_words[i], feature_names[new_replacements[i]][0]]
                )
            else:
                final_exp.append([output_removed_words[i], "---"])

        number_explanations = len(explanations)
        if np.size(explanations_score_change) > 1:
            inds = np.argsort(explanations_score_change, axis=0)
            inds = np.fliplr([inds])[0]
            inds_2 = []
            for i in range(np.size(inds)):
                inds_2.append(inds[i][0])
            explanation_set_adjusted = []
            for i in range(np.size(inds)):
                j = inds_2[i]
                explanation_set_adjusted.append(explanation_set[j])
            explanations_score_change_adjusted = []
            for i in range(np.size(inds)):
                j = inds_2[i]
                explanations_score_change_adjusted.append(explanations_score_change[j])
            explanation_set = explanation_set_adjusted
            explanations_score_change = explanations_score_change_adjusted

        time_elapsed = time.time() - tic
        print("\n Total elapsed time %d \n" % time_elapsed)

        indices_active_elements = np.nonzero(instance)[1]
        # Find the elements in indices_active_elements_explain that are not in indices_active_elements
        print("indices_active_elements", indices_active_elements)

        return {
            "final_exp": final_exp,
            "number active elements": number_active_elements,
            "number explanations found": number_explanations,
            "size smallest explanation": minimum_size_explanation,
            "time elapsed": time_elapsed,
            "differences score": explanations_score_change[0 : self.max_explained],
            "iterations": iteration,
            "final_sentence": final_sentence,
        }

In [ ]:
# Do no need
class SEDC_Explainer_no(object):
    """Class for generating evidence counterfactuals for classifiers on behavioral/text data"""

    def __init__(
        self,
        feature_names,
        classifier_fn,
        threshold_classifier,
        max_iter=100,
        max_explained=1,
        BB=True,
        max_features=30,
        time_maximum=120,
        revert=0,
    ):
        """Init function

        Args:
            classifier_fn: [function] classifier prediction probability function
            or decision function. For ScikitClassifiers, this is classifier.predict_proba
            or classifier.decision_function or classifier.predict_log_proba.
            Make sure the function only returns one (float) value. For instance, if you
            use a ScikitClassifier, transform the classifier.predict_proba as follows:

                def classifier_fn(X):
                    c=classification_model.predict_proba(X)
                    y_predicted_proba=c[:,1]
                    return y_predicted_proba

            threshold_classifier: [float] the threshold that is used for classifying
            instances as positive or not. When score or probability exceeds the
            threshold value, then the instance is predicted as positive.
            We have no default value, because it is important the user decides
            a good value for the threshold.

            feature_names: [numpy.array] contains the interpretable feature names,
            such as the words themselves in case of document classification or the names
            of visited URLs.

            max_iter: [int] maximum number of iterations in the search procedure.
            Default is set to 50.

            max_explained: [int] maximum number of EDC explanations generated.
            Default is set to 1.

            BB: [“True” or “False”]  when the algorithm is augmented with
            branch-and-bound (BB=True), one is only interested in the (set of)
            shortest explanation(s). Default is "True".

            max_features: [int] maximum number of features allowed in the explanation(s).
            Default is set to 30.

            time_maximum: [int] maximum time allowed to generate explanations,
            expressed in minutes. Default is set to 2 minutes (120 seconds).
        """

        self.feature_names = feature_names
        self.classifier_fn = classifier_fn
        self.threshold_classifier = threshold_classifier
        self.max_iter = max_iter
        self.max_explained = max_explained
        self.BB = BB
        self.max_features = max_features
        self.time_maximum = time_maximum
        self.revert = None
        self.initial_class = None

    def explanation(self, instance):
        """Generates evidence counterfactual explanation for the instance.
        ONLY IF THE CURRENT INSTANCE IS POSITIVE -> Limitation

        Args:
            instance: [numpy.array or sparse matrix] instance to explain

        Returns:
            A dictionary where:

                explanation_set: explanation(s) ranked from high to low change
                in predicted score or probability.
                The number of explanations shown depends on the argument max_explained.

                number_active_elements: number of active elements of
                the instance of interest.

                number_explanations: number of explanations found by algorithm.

                minimum_size_explanation: number of features in the smallest explanation.

                time_elapsed: number of seconds passed to generate explanation(s).

                explanations_score_change: change in predicted score/probability
                when removing the features in the explanation, ranked from
                high to low change.
        """

        # *** INITIALIZATION ***
        print("Start initialization...")
        tic = time.time()
        instance = lil_matrix(instance)
        iteration = 0
        nb_explanations = 0
        minimum_size_explanation = np.nan
        explanations = []
        explanations_sets = []
        explanations_score_change = []
        expanded_combis = []
        score_predicted = self.classifier_fn(instance)  ## Returns Prediction Prob
        # Intial class is 1 is score is greater than threshold
        if score_predicted > self.threshold_classifier:
            self.initial_class = [1]
        else:
            self.initial_class = [0]
            self.revert = 1
        print(
            "score_predicted  ",
            score_predicted,
            "  initial_class  ",
            self.initial_class,
        )

        reference = np.reshape(
            np.zeros(np.shape(instance)[1]), (1, len(np.zeros(np.shape(instance)[1])))
        )
        reference = sparse.csr_matrix(reference)

        explainer = shap.KernelExplainer(self.classifier_fn, reference, link="identity")
        shapVals = explainer.shap_values(instance, nsamples=5000, l1_reg="aic")

        features = []
        for ind in range(len(shapVals[0])):
            if shapVals[0, ind] != 0:
                features.append({"feature": ind, "shapValue": shapVals[0, ind]})
        sorted_data_in = sorted(features, key=lambda x: x["shapValue"], reverse=True)
        inverse_sorted_data_in = sorted(features, key=lambda x: x["shapValue"])

        if self.revert == 1:
            sorted_data_in = inverse_sorted_data_in

        indices_active_elements = np.nonzero(instance)[
            1
        ]  ## -> Gets non zero elements in the instance as an array [x, y, z]
        sorted_indices = sorted(
            indices_active_elements, key=lambda x: shapVals[0, x], reverse=True
        )
        indices_active_elements = np.array(sorted_indices)
        number_active_elements = len(indices_active_elements)
        indices_active_elements = indices_active_elements.reshape(
            (number_active_elements, 1)
        )  ## -> Reshape to get a predictable

        candidates_to_expand = (
            []
        )  # -> These combinations are further expanded -> These are the elements to be removed from the sentence
        for features in indices_active_elements:
            candidates_to_expand.append(OrderedSet(features))
        print("candidates_to_expand ", candidates_to_expand)
        ## > Gets an array with each element in reshaped incides as an ordered set -> [OrderedSet([430]), OrderedSet([588]), OrderedSet([595])]

        explanation_candidates = candidates_to_expand.copy()
        print("explanation_candidates ", explanation_candidates)
        ## Gets a copy of the above array -> Initially

        feature_set = [
            frozenset(x) for x in indices_active_elements
        ]  ## Immutable -> can be used as keys in dictionary
        ## Used features in the current x-reference -> incides of the words in the review.

        print("Initialization is complete.")
        print("\n Elapsed time %d \n" % (time.time() - tic))

        # *** WHILE LOOP ***
        while (
            (iteration < self.max_iter)
            and (nb_explanations < self.max_explained)
            and (len(candidates_to_expand) != 0)
            and (len(explanation_candidates) != 0)
            and ((time.time() - tic) < self.time_maximum)
        ):
            ## Stop if maximum iterations exceeded
            #  number of explanations generated is greater than the maximum explanations
            #  There are no candidates to expand
            #  There are no explanation candidates -> Used to force stop while loop below
            #  Or maximum allowed time exceeded
            iteration += 1
            print("\n Iteration %d \n" % iteration)

            if iteration == 1:
                print("Run in first iteration -> perturbation done \n")
                # Print the word in each index in the explanation candidates
                # for item in explanation_candidates:
                #     print([self.feature_names[x] for x in item])
                replacements = [
                    get_antonyms(self.feature_names[x[0]], loaded_plain_model_rf)
                    for x in explanation_candidates
                ]
                # convert each element in replacement to a OrderedSet
                replacements = [OrderedSet(x) for x in replacements]
                print("replacements \n", replacements, "\n")
                print("explanation_candidates \n", explanation_candidates, "\n")
                perturbed_instances = [
                    perturb_fn(x, inst=instance.copy()) for x in explanation_candidates
                ]
                replaced_instances = []
                for i in range(len(explanation_candidates)):
                    if replacements[i] == OrderedSet():
                        replaced_instances.append(
                            perturb_fn(
                                x=explanation_candidates[i], inst=instance.copy()
                            )
                        )
                    else:
                        replaced_instances.append(
                            replace_fn(
                                x=explanation_candidates[i],
                                y=replacements[i],
                                inst=instance.copy(),
                            )
                        )
                print("replaced_instances \n", replaced_instances, "\n")
                # Remove the elements in the indices given by the ordered set x and return an array fo such elements
                # Removes only one element in the first run -> Contains sentences with one word removed
                perturbed_instances = replaced_instances
                scores_explanation_candidates = [
                    self.classifier_fn(x, self.revert) for x in perturbed_instances
                ]
                # Get predictions for each perturbed instance where one or more elements are removed from the initial instance
                # It is in form of [[x], [y], [z]]
                print(
                    "scores_explanation_candidates \n",
                    scores_explanation_candidates,
                    "\n",
                )
                scores_candidates_to_expand = scores_explanation_candidates.copy()

            scores_perturbed_new_combinations = [
                x[0] for x in scores_explanation_candidates
            ]
            # Therefore get it to the shape [x, y, z] by getting the [0] th element of each element array
            # print(
            #     "scores_perturbed_new_combinations ", scores_perturbed_new_combinations
            # )

            # ***CHECK IF THERE ARE EXPLANATIONS***
            new_explanations = list(
                compress(
                    explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            # Get explanation candidates where their probability is less than the threshold classifier -> Positive becomes negative
            # print("New Explanations \n", new_explanations)
            explanations += list(
                compress(
                    explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            # print("\n explanations, explanations_score_change", explanations)
            nb_explanations += len(
                list(
                    compress(
                        explanation_candidates,
                        scores_perturbed_new_combinations < self.threshold_classifier,
                    )
                )
            )  # Update number of explanations which pass the required threshold
            explanations_sets += list(
                compress(
                    explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            explanations_sets = [
                set(x) for x in explanations_sets
            ]  # Convert each array to a set -> to get the words
            explanations_score_change += list(
                compress(
                    scores_explanation_candidates,
                    scores_perturbed_new_combinations < self.threshold_classifier,
                )
            )
            # print('explanations_score_change', explanations_score_change)

            # Adjust max_length
            if self.BB == True:
                if len(explanations) != 0:
                    lengths = []  # Record length of each explanation found
                    for explanation in explanations:
                        lengths.append(len(explanation))
                    lengths = np.array(lengths)
                    max_length = lengths.min()
                    # Get minimum length of the found explanations as max length -> Do not search for explanations with longer length
                else:
                    max_length = number_active_elements  # Else can find maximum length equal to number of words in instance
            else:
                max_length = number_active_elements
            print("\n-------------Max length updated to - ", max_length)

            # Eliminate combinations from candidates_to_expand ("best-first" candidates) that can not be expanded
            # Pruning based on Branch & Bound=True, max. features allowed and number of active features
            candidates_to_expand_updated = []
            scores_candidates_to_expand_updated = (
                []
            )  # enumerate -> Find count of || to list one after another
            for j, combination in enumerate(candidates_to_expand):
                if (
                    (len(combination) < number_active_elements)
                    and (len(combination) < max_length)
                    and (len(combination) < self.max_features)
                ):
                    # Combination length should be less than the words in the input and max length of the required explanation and required maximum features
                    candidates_to_expand_updated.append(
                        combination
                    )  # If the combination matches, it is further expanded
                    scores_candidates_to_expand_updated.append(
                        scores_candidates_to_expand[j]
                    )
                    # Add the prediction score to the new array
                    # get the score from the scores_candidates_to_expand using the current index

            print(
                "\nlen(candidates_to_expand_updated)",
                len(candidates_to_expand_updated),
                " 0 ",
            )
            print(
                "\nnb_explanations",
                nb_explanations,
                " >= self.max_explained ",
                self.max_explained,
            )

            # *** IF LOOP ***
            # expanding the candidates to update will exceed the max length set in the earlier loop
            if (len(candidates_to_expand_updated) == 0) or (
                nb_explanations >= self.max_explained
            ):
                ## If the number of explanations exceeded the required number
                ## or no candidates
                ## no explanations present

                print("nb_explanations Stop iterations...")
                explanation_candidates = []  # stop algorithm
                ## Found all the candidates
                print(
                    "scores_candidates_to_expand_updated  ",
                    scores_candidates_to_expand_updated,
                )
                # print("candidates_to_expand_updated   ", candidates_to_expand_updated)

            elif len(candidates_to_expand_updated) != 0:
                ## If there are possible candidates
                print("elif", len(candidates_to_expand_updated), " != 0")

                explanation_candidates = []
                it = 0  # Iteration of the while loop
                indices = []

                scores_candidates_to_expand2 = []
                for score in scores_candidates_to_expand_updated:
                    if score[0] < self.threshold_classifier:
                        scores_candidates_to_expand2.append(2 * score_predicted)
                    else:
                        scores_candidates_to_expand2.append(score)
                # update candidate scores if they have score less than threshold -> To expand them further
                shap_candidates_to_expand2 = []
                for candidate in candidates_to_expand_updated:
                    shapValues = 0
                    for word in candidate:
                        # find word in feature column in sorted_data
                        for ind in range(len(sorted_data_in)):
                            if sorted_data_in[ind]["feature"] == word:
                                shapValues += sorted_data_in[ind]["shapValue"]
                                break
                    shap_candidates_to_expand2.append(shapValues)

                # print(
                #     "\n scores_candidates_to_expand2 before loop",
                #     scores_candidates_to_expand2,
                # )

                # *** WHILE LOOP ***
                while (
                    (len(explanation_candidates) == 0)
                    and (it < len(scores_candidates_to_expand2))
                    and ((time.time() - tic) < self.time_maximum)
                ):
                    # Stop if candidates are found or looped through more than there are candidates or maximum time reached

                    print("While loop iteration %d" % it)

                    if it != 0:  # Because indices are not there in the first iteration
                        for index in indices:
                            scores_candidates_to_expand2[index] = 2 * score_predicted

                    # print(
                    #     "\n scores_candidates_to_expand2 after loop",
                    #     scores_candidates_to_expand2,
                    # )
                    # print("\n indices", indices)

                    # do elementwise subtraction between score_predicted and scores_candidates_to_expand2
                    subtractionList = []
                    for x, y in zip(score_predicted, scores_candidates_to_expand2):
                        print("\n x, y", x - y)
                        subtractionList.append(x - y)

                    # Do element wise subtraction between the prediction score of the x_ref and every element of the scores_candidates_to_expand2
                    index_combi_max = np.argmax(subtractionList)
                    if self.revert == 0:
                        index_combi_max = np.argmax(subtractionList)
                    else:
                        index_combi_max = np.argmin(subtractionList)
                    # index_shap_max = np.argmax(shap_candidates_to_expand2)
                    # index_shap_min = np.argmin(shap_candidates_to_expand2)
                    # if self.revert == 0:
                    #     index_combi_max = index_shap_max
                    # else:
                    #     index_combi_max = index_shap_min
                    # print(
                    #     "subtrac max ",
                    #     index_combi_max,
                    #     " index_shap_max ",
                    #     index_shap_max,
                    # )
                    # # Get the index of the maximum value -> Expand it
                    # print(
                    #     "\n index_combi_max",
                    #     candidates_to_expand_updated[np.argmax(subtractionList)],
                    #     "\n index_shap_max",
                    #     candidates_to_expand_updated[index_combi_max],
                    # )
                    indices.append(index_combi_max)
                    expanded_combis.append(
                        candidates_to_expand_updated[index_combi_max]
                    )
                    # Add this combination to already expanded combinations as it will be expanded next by expand and prune function

                    comb_to_expand = candidates_to_expand_updated[index_combi_max]
                    # Expand the found combination with highest difference
                    func = expand_and_prune(
                        comb_to_expand,
                        expanded_combis,
                        feature_set,
                        candidates_to_expand_updated,
                        explanations_sets,
                        scores_candidates_to_expand_updated,
                        instance,
                        self.classifier_fn,
                        self.revert,
                    )
                    """Returns:
                        - explanation_candidates: combinations of features that are explanation
                        candidates to be checked in the next iteration
                        - candidates_to_expand: combinations of features that are candidates to
                        expanded in next iterations or candidates for "best-first"
                        - expanded_combis: [list] list of combinations of features that are already
                        expanded as "best-first"
                        - scores_candidates_to_expand: scores after perturbation for the candidate
                        combinations of features to be expanded
                        - scores_explanation_candidates: scores after perturbation of explanation candidates"""
                    explanation_candidates = func[0]
                    candidates_to_expand = func[1]
                    expanded_combis = func[2]
                    scores_candidates_to_expand = func[3]
                    scores_explanation_candidates = func[4]

                    it += 1

                print(
                    "\n\n\niteration - ", iteration, " self.max_iter - ", self.max_iter
                )
                print(
                    "\n\nlen(candidates_to_expand) - ",
                    len(candidates_to_expand),
                    " != 0 ",
                )
                print(
                    "\n\nlen(explanation_candidates) - ",
                    len(explanation_candidates),
                    " !=0 ",
                )
                print(
                    "\n\n(time.time() - tic) - ",
                    (time.time() - tic),
                    " self.time_maximum - ",
                    self.time_maximum,
                )
            print("\n Elapsed time %d \n" % (time.time() - tic))

        # *** FINAL PART OF ALGORITHM ***
        print("Iterations are done.")

        explanation_set = []
        explanation_feature_names = []
        for i in range(len(explanations)):
            explanation_feature_names = []
            for features in explanations[i]:
                explanation_feature_names.append(self.feature_names[features])
            explanation_set.append(explanation_feature_names)

        if len(explanations) != 0:
            lengths_explanation = []
            for explanation in explanations:
                l = len(explanation)
                lengths_explanation.append(l)
            minimum_size_explanation = np.min(lengths_explanation)

        number_explanations = len(explanations)
        if np.size(explanations_score_change) > 1:
            inds = np.argsort(explanations_score_change, axis=0)
            inds = np.fliplr([inds])[0]
            inds_2 = []
            for i in range(np.size(inds)):
                inds_2.append(inds[i][0])
            explanation_set_adjusted = []
            for i in range(np.size(inds)):
                j = inds_2[i]
                explanation_set_adjusted.append(explanation_set[j])
            explanations_score_change_adjusted = []
            for i in range(np.size(inds)):
                j = inds_2[i]
                explanations_score_change_adjusted.append(explanations_score_change[j])
            explanation_set = explanation_set_adjusted
            explanations_score_change = explanations_score_change_adjusted

        time_elapsed = time.time() - tic
        print("\n Total elapsed time %d \n" % time_elapsed)

        print(
            "If we remove the words ",
            explanation_set[0 : self.max_explained],
            "From the review, the prediction will be reversed",
        )

        return {
            "explanation set": explanation_set[0 : self.max_explained],
            "number active elements": number_active_elements,
            "number explanations found": number_explanations,
            "size smallest explanation": minimum_size_explanation,
            "time elapsed": time_elapsed,
            "differences score": explanations_score_change[0 : self.max_explained],
            "iterations": iteration,
        }

In [ ]:
# Get threshold_classifier_probs
p = np.sum(y_train_imdb) / np.size(y_train_imdb)

probs = loaded_plain_model_lr.predict(x_test_imdb)
threshold_classifier_probs = np.percentile(probs, (50.41))
print(threshold_classifier_probs)
predictions_probs = probs >= threshold_classifier_probs

accuracy_test = accuracy_score(y_test_imdb, np.array(predictions_probs))
print("The accuracy of the model on the test data is %f" % accuracy_test)

# indices_probs_pos = np.nonzero(predictions_probs)

In [ ]:
# Do not need
for i in range(x_test_imdb.shape[0]):
    counter = 0
    if round(classifier_fn_lr(x_test_imdb[i, :])[0], 1) == 0.6:
        counter = counter + 1
        print(i)
    if counter > 10:
        break

In [6]:
indices_arr = []
for index in range(100):
    score = classifier_fn_lr(x_test_imdb[index, :])[0]
    if score < 0.9 and score > 0.8:
        indices_arr.append(index)
        print(index, score)

13 0.8710484661364114
18 0.8183767362908165
34 0.8526536137479714
53 0.8851770198684602
60 0.8385767256610298
63 0.8927611936112447
64 0.8666939779029769
70 0.8170098160732301


In [ ]:
explainer_shap = SEDC_Explainer(
    feature_names=feature_names,
    threshold_classifier=threshold_classifier_probs,
    classifier_fn=classifier_fn_lr,
    max_iter=50,
    time_maximum=120,
)


explanation_normal = explainer_shap.explanation(x_test_imdb[10, :])